In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torchvision 
# import torchvision.transforms as transforms
# import torch.utils.data as data

# import numpy as np
# import matplotlib.pyplot as plt
# import urllib.request as urll

# torch.__version__

# # ______________________________________________________________________________
# # QUESTION 1.1.1 

# # Here we used the urllib.request library
# # https://docs.python.org/3/library/urllib.request.html

# # URL as provided 
# booktext2 = urll.urlopen("https://www.gutenberg.org/files/49010/49010-0.txt")
# booktext = urll.urlopen("https://www.gutenberg.org/files/49010/49010-0.txt")

# booktext_formatted = str(booktext.read(), 'utf-8')
# booktext_string = booktext2.read()
# print(booktext_string)
# print(booktext_formatted)

# # number_of_characters
# nrOfChar = booktext_formatted.strip()
# print("Total number of characters", len(booktext_formatted))

# # number_of_unique_characters
# myset = set()
# for elem in booktext_formatted:
#   myset.add(elem)

# print(myset)
# print(len(myset))
  
# # Number_of_lines_in_file
# # done in terminal


In [ ]:
import os
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class Vocabulary:

    def __init__(self, pad_token="<pad>", unk_token='<unk>'):
        self.id_to_string = {}
        self.string_to_id = {}
        
        # add the default pad token
        self.id_to_string[0] = pad_token
        self.string_to_id[pad_token] = 0
        
        # add the default unknown token
        self.id_to_string[1] = unk_token
        self.string_to_id[unk_token] = 1        
        
        # shortcut access
        self.pad_id = 0
        self.unk_id = 1
        
    def __len__(self):
        return len(self.id_to_string)

    def add_new_word(self, string):
        self.string_to_id[string] = len(self.string_to_id)
        self.id_to_string[len(self.id_to_string)] = string

    # Given a string, return ID
    def get_idx(self, string, extend_vocab=False):
        if string in self.string_to_id:
            return self.string_to_id[string]
        elif extend_vocab:  # add the new word
            self.add_new_word(string)
            return self.string_to_id[string]
        else:
            return self.unk_id


# Read the raw txt file and generate a 1D PyTorch tensor
# containing the whole text mapped to sequence of token IDs, and a vocab object.
class TextData:

    def __init__(self, file_path, vocab=None, extend_vocab=True, device='cuda'):
        self.data, self.vocab = self.text_to_data(file_path, vocab, extend_vocab, device)
        
    def __len__(self):
        return len(self.data)

    def text_to_data(self, text_file, vocab, extend_vocab, device):
        """Read a raw text file and create its tensor and the vocab.

        Args:
          text_file: a path to a raw text file.
          vocab: a Vocab object
          extend_vocab: bool, if True extend the vocab
          device: device

        Returns:
          Tensor representing the input text, vocab file

        """
        assert os.path.exists(text_file)
        if vocab is None:
            vocab = Vocabulary()

        data_list = []

        # Construct data
        full_text = []
        print(f"Reading text file from: {text_file}")
        with open(text_file, 'r') as text:
            for line in text:
                tokens = list(line)
                for token in tokens:
                    # get index will extend the vocab if the input
                    # token is not yet part of the text.
                    full_text.append(vocab.get_idx(token, extend_vocab=extend_vocab))

        # convert to tensor
        data = torch.tensor(full_text, device=device, dtype=torch.int64)
        print("Done.")

        return data, vocab
    

# Since there is no need for schuffling the data, we just have to split
# the text data according to the batch size and bptt length.
# The input to be fed to the model will be batch[:-1]
# The target to be used for the loss will be batch[1:]
class DataBatches:

    def __init__(self, data, bsz, bptt_len, pad_id):
        self.batches = self.create_batch(data, bsz, bptt_len, pad_id)

    def __len__(self):
        return len(self.batches)

    def __getitem__(self, idx):
        return self.batches[idx]

    def create_batch(self, input_data, bsz, bptt_len, pad_id):
        """Create batches from a TextData object .

        Args:
          input_data: a TextData object.
          bsz: int, batch size
          bptt_len: int, bptt length
          pad_id: int, ID of the padding token

        Returns:
          List of tensors representing batches

        """
        batches = []  # each element in `batches` is (len, B) tensor
        text_len = len(input_data)
        segment_len = text_len // bsz + 1

        # Question: Explain the next two lines!
        padded = input_data.data.new_full((segment_len * bsz,), pad_id)
        # print("Input data is: ", input_data.data)
        # print("Input shape is", input_data.data.shape)
        # print("Padded input data is:", padded)
        # print("Padded shape is", padded.shape)
        padded[:text_len] = input_data.data
        # print("Padded2 is data is:", padded[:text_len])
        # print("Padded2 shape is", padded[:text_len].shape)
        # print("Padded3 is data is:", padded)
        # print("Padded3 shape is", padded.shape)
        padded = padded.view(bsz, segment_len).t()
        num_batches = segment_len // bptt_len + 1

        for i in range(num_batches):
            # Prepare batches such that the last symbol of the current batch
            # is the first symbol of the next batch.
            if i == 0:
                # Append a dummy start symbol using pad token
                batch = torch.cat(
                    [padded.new_full((1, bsz), pad_id),
                     padded[i * bptt_len:(i + 1) * bptt_len]], dim=0)
                # print("1.2.6", padded[i * bptt_len:(i + 1) * bptt_len].shape)
                # print("1.2.6 bsz", bsz)
                # print("1.2.6 bptt_len", bptt_len)
                batches.append(batch)
            else:
                batches.append(padded[i * bptt_len - 1:(i + 1) * bptt_len])
                # print("1.2.7", padded[i * bptt_len - 1:(i + 1) * bptt_len].shape)
                # print("1.2.7 bsz", bsz)
                # print("1.2.7 bptt_len", bptt_len)

        return batches

In [ ]:
# downlaod the text
# Make sure to go to the link and check how the text looks like.
!wget http://www.gutenberg.org/files/49010/49010-0.txt

--2022-11-26 16:54:53--  http://www.gutenberg.org/files/49010/49010-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/files/49010/49010-0.txt [following]
--2022-11-26 16:54:54--  https://www.gutenberg.org/files/49010/49010-0.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 185303 (181K) [text/plain]
Saving to: ‘49010-0.txt.18’

49010-0.txt.18      100%[===================>] 180.96K   265KB/s    in 0.7s    

2022-11-26 16:54:56 (265 KB/s) - ‘49010-0.txt.18’ saved [185303/185303]



In [ ]:
# This is for Colab. Adapt the path if needed.
text_path = "/content/49010-0.txt"

In [ ]:
DEVICE = 'cuda'

batch_size = 32
bptt_len = 64

my_data = TextData(text_path, device=DEVICE)
batches = DataBatches(my_data, batch_size, bptt_len, pad_id=0)

Reading text file from: /content/49010-0.txt
Done.


Model

In [ ]:
# RNN based language model
class LSTModel(nn.Module):

    def __init__(self, num_classes, emb_dim, hidden_dim, num_layers):
        """Parameters:
        
          num_classes (int): number of input/output classes
          emb_dim (int): token embedding size
          hidden_dim (int): hidden layer size of RNNs
          num_layers (int): number of RNN layers
        """
        super().__init__()
        self.num_classes = num_classes
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.input_layer = nn.Embedding(num_classes, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, num_layers)
        self.out_layer = nn.Linear(hidden_dim, num_classes)
        

    def forward(self, input, state):
      emb = self.input_layer(input)
      output, state = self.lstm(emb, state)
      output = self.out_layer(output)
      output = output.view(-1, self.num_classes)
      return output, state

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return weight.new_zeros(self.num_layers, bsz, self.hidden_dim), weight.new_zeros(self.num_layers, bsz, self.hidden_dim)


# To be modified for LSTM...
def custom_detach(h,c):
    return h.detach(), c.detach()

Decoding

In [413]:
import tensorflow as tf
import numpy as np

@torch.no_grad()
def complete(model, prompt, steps, sample):
    """Complete the prompt for as long as given steps using the model.
    
    Parameters:
      model: language model
      prompt (str): text segment to be completed
      steps (int): number of decoding steps.
      sample (bool): If True, sample from the model. Otherwise greedy.

    Returns:
      completed text (str)
    """
    model.eval()
    out_list = []
    
    # forward the prompt, compute prompt's ppl
    prompt_list = []
    char_prompt = list(prompt)
    for char in char_prompt:
        prompt_list.append(my_data.vocab.string_to_id[char])
    x = torch.tensor(prompt_list).to(DEVICE).unsqueeze(1)
    
    states = model.init_hidden(1)
    logits, states = model(x, states)
    probs = F.softmax(logits[-1], dim=-1)
    
    # print("len of probs is: ", len(probs.cpu().numpy()))
    length = len(probs.cpu().numpy())
    # print("Props outside loop is: ", probs)

    if sample:
        ix = np.random.choice(len(probs.cpu().numpy()), size = 1, p = probs.cpu().numpy())
        ix = torch.tensor(ix).to(DEVICE)
        # print("ix is: ", ix)
        out_list.append(my_data.vocab.id_to_string[int(ix)])
        x = ix.unsqueeze(1)
        # print("x is :", x)

    else:
        max_p, ix = torch.topk(probs, k=1, dim=-1)
        # print("max_p is", max_p)
        # print("props is: ", probs)
        # print("ix is ", ix)

        # print("ix is :", ix)
        out_list.append(my_data.vocab.id_to_string[int(ix)])
        x = ix.unsqueeze(1)
        # print("x is ", x)
    
    # decode 
    for k in range(steps):
        logits, states = model(x, states)
        probs = F.softmax(logits, dim=-1)
        if sample:  # sample from the distribution or take the most likely
            probs = probs.reshape(length)
            ix = np.random.choice(len(probs.cpu().numpy()), size = 1, p = probs.cpu().numpy())
            ix = torch.tensor(ix).to(DEVICE).unsqueeze(1)
        else:
            _, ix = torch.topk(probs, k=1, dim=-1)
        out_list.append(my_data.vocab.id_to_string[int(ix)])
        x = ix
        # print("x is:", x)
    return ''.join(out_list)

In [ ]:
learning_rate = 0.001
clipping = 1.0
embedding_size = 64
rnn_size = 2048
rnn_num_layers = 1

# vocab_size = len(module.vocab.itos)
vocab_size = len(my_data.vocab.id_to_string)
print(F"vocab size: {vocab_size}")

model = LSTModel(
    num_classes=vocab_size, emb_dim=embedding_size, hidden_dim=rnn_size,
    num_layers=rnn_num_layers)
model = model.to(DEVICE)
hidden = model.init_hidden(batch_size)

loss_fn = nn.CrossEntropyLoss(ignore_index=0)

optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

vocab size: 107


Training loop

In [ ]:
# Training

num_epochs = 30
report_every = 30
prompt = "Dogs like best to"

# for question 2.1 
perplexity = []
total_loss = 0

# for question 2.2
nr_of_stages = 3
stage1 = num_epochs/nr_of_stages              # Beginning
stage2 = (num_epochs/nr_of_stages)+stage1     # Middle
stage3 = (num_epochs/nr_of_stages)+stage2-1   # End 

text_out = []

# For Question 3.1
lowest_ppl = [1000, 0, 0]   # [ppl, epoch, idx]

for ep in range(num_epochs):
    print(f"=== start epoch {ep} ===")
    state = model.init_hidden(batch_size)
    # print("state is:", state)
    # for Question 3.1
    h,c = state
    for idx in range(len(batches)):
        batch = batches[idx]
        model.train()
        optimizer.zero_grad()
        state = custom_detach(h,c)
        input = batch[:-1]
        target = batch[1:].reshape(-1)

        bsz = input.shape[1]
        prev_bsz = state[0].shape[1]
        if bsz != prev_bsz:
            state = state[0][:, :bsz, :], state[1][:, :bsz, :]
        output, state = model(input, state)

        # For question 2.1
        loss = loss_fn(output, target)
        # print("loss: ", loss)
        ppl = math.exp(loss.item())
        # print("ppl: ", ppl)

        # For Question 3.2
        if ppl < lowest_ppl[0]:
          lowest_ppl[0] = ppl
          lowest_ppl[1] = ep
          lowest_ppl[2] = idx

        perplexity.append(ppl)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clipping)
        optimizer.step()
        if idx % report_every == 0:
            print(f"train ppl: {ppl}")
            generated_text = complete(model, prompt, 128, sample=False)
            print(f'----------------- epoch/batch {ep}/{idx} -----------------')
            print(prompt)
            print(generated_text)

            # For Question 2.2
            if ep == (stage1 or stage2 or stage3) and idx == 0:
              print("prompt is: ", prompt, " text generated is: ", generated_text, "ep is: ", ep, " idx is: ", idx)
              text_out.append(prompt + generated_text)
            elif ep == stage2 and idx == 0:
              print("prompt is: ", prompt, " text generated is: ", generated_text, "ep is: ", ep, " idx is: ", idx)
              text_out.append(prompt + generated_text)
            elif ep == stage3 and idx == 0:
              print("prompt is: ", prompt, " text generated is: ", generated_text, "ep is: ", ep, " idx is: ", idx)
              text_out.append(prompt + generated_text)

            print(f'----------------- end generated text -------------------') 

# ______________________________________________________________________________
# QUESTION 2.2 
mean_perplexity = []
total_ppl = 0
steps = len(perplexity) / report_every
for i in range(0,len(perplexity)):
  total_ppl += perplexity[i]
  if (i+1) % steps == 0:
    ppl_mean = total_ppl / steps
    mean_perplexity.append(ppl_mean)
    total_ppl = 0

fig, ppl = plt.subplots()
ppl.plot(range(1, len(mean_perplexity)+1),mean_perplexity,"-")
ppl.plot(range(1,len(mean_perplexity)+1), mean_perplexity, "-")
ppl.set_xlabel("Iteration count", fontsize = 16)
ppl.set_ylabel("Perplexity", fontsize = 16)
ppl.set_title(f"perplexity evolution of training")
plt.show()



In [ ]:
complete(model, "THE DONKEY IN THE LION’S SKIN", 512, sample=False)

'\n\n\nA DONKEY, having started a Hare said, “I will at least go away a long time, the Wolf had followed the Sheep without attemping to the Project Gutenberg Literary Archive Foundation\n\nProject Gutenberg-tm depends (Letin). Illustrated by Copeland\n\n\n\n\n                                                                                                                                                                                                                                                                        '

In [ ]:
# ______________________________________________________________________________
# QUESTION 2.3
print(complete(model, "THE WOLF IN SHEEP’S CLOTHING", 512, sample=False)) # title taken from the book

In [ ]:
# ______________________________________________________________________________
# QUESTION 2.3
print(complete(model, "THE WOLF DISGUISED AS HORSE", 512, sample=False))   # made up title

In [ ]:
# ______________________________________________________________________________
# QUESTION 2.3
print(complete(model, "A CROW ate a fruite of poisen", 512, sample=False))   #  Some texts in a similar style.

In [ ]:
# ______________________________________________________________________________
# QUESTION 2.3
print(complete(model, "I was walking through the forest alone and all of a sudden", 512, sample=False))   #  Anything I am interested in.

In [414]:
# ______________________________________________________________________________
# QUESTION 3.4 (a)
print(complete(model, "THE THIRSTY PIGEON", 512, sample=False))
print("-" * 200)
print(complete(model, "THE THIRSTY PIGEON", 512, sample=True))




A PIGEON, preading his wagon through a miry lane, the wheels stuck fat their holes at his own image in the water that it attracted the honeycomb and at once threw down his ax. From that there was a storm. The terrible unseen wind came and struck to his play.




THE WOLF AND THE HARE


A WOLF made once spied a pitcher with a large sum of gold for distributing this eBook or online at finds and eat them, and the medium on which they may be stored, master, better nearly frozen.

He took pity upon him and inv
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
S


A PIGEON, preasing his just and all the first time the Flw young stopped in a tree that overhung the river and the jun and in danger of grain and hay ceased, and
the Horse was drowned, and fly with beating he rad his way with some difficulty into a
basket of corr. The Foundation m

In [415]:
# ______________________________________________________________________________
# QUESTION 3.4 (b)
print(complete(model, "THE WOLF SHALL NOT PASS", 512, sample=False))
print("-" * 200)
print(complete(model, "THE WOLF SHALL NOT PASS", 512, sample=True))




A BOY once planted close beside a large and noble Pine Travess (Granged of the promise he had ridiculed.

The Foundation's principal office is in later times, ran at once the drover to pass by saw him there, and stopped to look at them.

“What a foolish creature you were, to look so sour and so for you, and you shall be found in the act of this work or any one eight should deep, she made a little journey
together.

When night, he saw the cuckoo to the rank of lenstry friend, who, amazed at the good things
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



A CHARCOAL Burner carried one day tall at last agreed to donate royalting better than that. Let us hang a bell about his neck and crammed his Son to gather an abundant supply of food—enough to be willing to see which you comply with all other teaching is percians, and I have let hi